In [1]:
import os
import xml.etree.ElementTree as ET
import pdb


In [2]:
def build_classification_tree(node):
    #pdb.set_trace()
    classification_symbol = node.find('classification-symbol').text
    title_text_list=[]
    cpc_text_list=[]
    title_parts = node.findall('class-title/title-part')
    for title_part in title_parts:
        text_tag = title_part.find('text')
        if text_tag is not None:
            text= text_tag.text
            title_text_list.append(text)
            
    cpc_text = node.findall('class-title/title-part/CPC-specific-text')
    for cpc_part in cpc_text:
        text_tag = cpc_part.find('text')
        if text_tag is not None:
            text= text_tag.text
            cpc_text_list.append(text)

    classification_def = title_text_list + cpc_text_list
    classification_def = [def_item for def_item in classification_def if def_item is not None]
    classification_def=';'.join(classification_def)
    children = node.findall('classification-item')

    if children:
        child_results = [build_classification_tree(child) for child in children]
        return (classification_symbol, classification_def, child_results)
    else:
        return (classification_symbol, classification_def)


In [3]:
# Function to process XML files in a folder
def process_xml_files_in_folder(folder_path):
    #pdb.set_trace()
    classification_trees = []
    for filename in os.listdir(folder_path):
        if filename.endswith('.xml'):
            xml_file_path = os.path.join(folder_path, filename)
            tree = ET.parse(xml_file_path)
            root = tree.getroot()
            baseroot = root[0]  # Assuming the structure is consistent
            classification_tree = build_classification_tree(baseroot)
            #print(classification_tree)
            classification_trees.append((filename, classification_tree))
    return classification_trees

In [4]:
# Full path to folder containing XML files
folder_path1 = os.path.join(os.path.expanduser("~"), "Downloads", "CPCSchemeXML202401")
folder_path2 = os.path.join(os.path.expanduser("~"), "Downloads", "CPCSchemeXML202401","ClassXMLfiles")
# Process XML files in the folder
classification_trees = process_xml_files_in_folder(folder_path1)
c_tree=process_xml_files_in_folder(folder_path2)

# Print classification trees
#classification_trees

In [5]:
import pdb

def get_child(child_list, prev_symbols, result):
    #pdb.set_trace()
    
    for child_tuple in child_list:
        symbol = child_tuple[0]
        definition = child_tuple[1]
        result[symbol] = prev_symbols + [definition]
        
        if len(child_tuple) == 3:
            sub_child = child_tuple[2]
            get_child(sub_child, prev_symbols + [definition], result)

    return result


In [6]:
withSlashDict = {}

In [7]:

for i in range(len(c_tree)):
    #child_list_example =[classification_trees[i][1]]
    child_list_example1 =[c_tree[i][1]]
    
    # Calling the function
    get_child(child_list_example1, [], withSlashDict)
    


In [8]:

for i in range(len(classification_trees)):
    prev=classification_trees[i][0].replace('.xml','')[-4:-1]
    child_list_example =[classification_trees[i][1]]
    #print(prev_sym)
    # Calling the function
    get_child(child_list_example, withSlashDict[prev], withSlashDict)
    


In [9]:
withSlashDict

{'A': ['HUMAN NECESSITIES'],
 'A01': ['HUMAN NECESSITIES',
  'AGRICULTURE',
  'AGRICULTURE;FORESTRY;ANIMAL HUSBANDRY;HUNTING;TRAPPING;FISHING'],
 'A01B': ['HUMAN NECESSITIES',
  'AGRICULTURE',
  'AGRICULTURE;FORESTRY;ANIMAL HUSBANDRY;HUNTING;TRAPPING;FISHING',
  'SOIL WORKING IN AGRICULTURE OR FORESTRY;PARTS, DETAILS, OR ACCESSORIES OF AGRICULTURAL MACHINES OR IMPLEMENTS, IN GENERAL '],
 'A01C': ['HUMAN NECESSITIES',
  'AGRICULTURE',
  'AGRICULTURE;FORESTRY;ANIMAL HUSBANDRY;HUNTING;TRAPPING;FISHING',
  'PLANTING;SOWING;FERTILISING '],
 'A01D': ['HUMAN NECESSITIES',
  'AGRICULTURE',
  'AGRICULTURE;FORESTRY;ANIMAL HUSBANDRY;HUNTING;TRAPPING;FISHING',
  'HARVESTING;MOWING'],
 'A01F': ['HUMAN NECESSITIES',
  'AGRICULTURE',
  'AGRICULTURE;FORESTRY;ANIMAL HUSBANDRY;HUNTING;TRAPPING;FISHING',
  'PROCESSING OF HARVESTED PRODUCE;HAY OR STRAW PRESSES;DEVICES FOR STORING AGRICULTURAL OR HORTICULTURAL PRODUCE '],
 'A01G': ['HUMAN NECESSITIES',
  'AGRICULTURE',
  'AGRICULTURE;FORESTRY;ANIMAL HUSBAN

In [10]:

for key, value_list in withSlashDict.items():
    filtered_list = [value for value in value_list if value != '']
    withSlashDict[key] = filtered_list

In [11]:
withSlashDict['A01B1/12']

['HUMAN NECESSITIES',
 'AGRICULTURE',
 'AGRICULTURE;FORESTRY;ANIMAL HUSBANDRY;HUNTING;TRAPPING;FISHING',
 'SOIL WORKING IN AGRICULTURE OR FORESTRY;PARTS, DETAILS, OR ACCESSORIES OF AGRICULTURAL MACHINES OR IMPLEMENTS, IN GENERAL ',
 'Hand tools ',
 'Hoes;Hand cultivators ',
 'with blades provided with teeth']

In [12]:
import re
withoutSlashDict={}
for key,val in withSlashDict.items():
    new_key=re.sub(r'[^a-zA-Z0-9]','',key)
    withoutSlashDict[new_key]=val


In [13]:
withoutSlashDict['C07D49804']

['CHEMISTRY;METALLURGY',
 'CHEMISTRY',
 'ORGANIC CHEMISTRY',
 'HETEROCYCLIC COMPOUNDS ',
 'Heterocyclic compounds containing condensed hetero ring systems',
 'Heterocyclic compounds containing in the condensed system at least one hetero ring having nitrogen and oxygen atoms as the only ring hetero atoms ',
 'in which the condensed system contains two hetero rings',
 'Ortho-condensed systems']

In [14]:
ResDefDict={}
for key,val in withSlashDict.items():
    ResDefDict[key]=val[-1]

In [15]:
ResDefDict['G04B1/00']

'Driving mechanisms '

In [ ]:
import json

def create_folders(str):
    folder_path="C:\\Users\\pooja\\Downloads\\JSONFiles\\" + str
    os.makedirs(folder_path,exist_ok=True)
    return folder_path

for key,val in withSlashDict.items():
    str=key[0]
    valdef=val
    keywithoutSlash=re.sub(r'[^a-zA-Z0-9]','',key)
    file_path=create_folders(str)+"\\"+ keywithoutSlash+ ".json"
    jsondata={
        key:valdef,
        keywithoutSlash:valdef,
        "last_def":val[-1], 
    }
    with open(file_path,"w") as jsonfile:
        json.dump(jsondata,jsonfile,indent=4)
    
    
    

In [18]:
import json

def create_folders(str):
    folder_path="C:\\Users\\pooja\\Downloads\\JSONFiles\\" + str
    os.makedirs(folder_path,exist_ok=True)
    return folder_path

for key,val in withSlashDict.items():
    str=key[0]
    valdef='/ '.join(val)
    keywithoutSlash=re.sub(r'[^a-zA-Z0-9]','',key)
    file_path=create_folders(str)+"\\"+ keywithoutSlash+ ".json"
    jsondata={
        'keyWithSlash':key,
        'keywithoutSlash':keywithoutSlash,
        'Definition':valdef,
        "last_def":val[-1], 
    }
    with open(file_path,"w") as jsonfile:
        json.dump(jsondata,jsonfile,indent=4)
    
    
    

In [ ]:
xml_path="Downloads/CPCSchemeXML202401/cpc-scheme-A01B.xml"

In [ ]:
tree=ET.parse(xml_path)

In [ ]:
root=tree.getroot()

In [ ]:
root.tag

In [ ]:
root.findall('classification-item')

In [ ]:
baseroot=root[0]
baseroot

In [ ]:
def build_classification_tree(node):
    #pdb.set_trace()
    classification_symbol = node.find('classification-symbol').text
    title_text_list=[]
    cpc_text_list=[]
    title_parts = node.findall('class-title/title-part')
    for title_part in title_parts:
        text_tag = title_part.find('text')
        if text_tag is not None:
            text= text_tag.text
            title_text_list.append(text)
            
    cpc_text = node.findall('class-title/title-part/CPC-specific-text')
    for cpc_part in cpc_text:
        text_tag = cpc_part.find('text')
        if text_tag is not None:
            text= text_tag.text
            cpc_text_list.append(text)

    classification_def = title_text_list + cpc_text_list
    classification_def=';'.join(classification_def)
    children = node.findall('classification-item')

    if children:
        child_results = [build_classification_tree(child) for child in children]
        return (classification_symbol, classification_def, child_results)
    else:
        return (classification_symbol, classification_def)


In [ ]:
classification_tree=build_classification_tree(baseroot)

In [ ]:
classification_tree


In [ ]:
prev=classification_trees[0][0].replace('.xml','')[-4:]
prev

In [ ]:
dict={'A':["hiii"]}
dict

In [ ]:
prev=dict['A']

In [ ]:
prev+["hello"]

In [ ]:
# Function to search for a value in a hierarchical list
def search_nested_list(nested_list, value_to_find):
    for item in nested_list:
        if isinstance(item, list):  # If the item is a sublist
            # Recursively search the sublist
            found = search_nested_list(item, value_to_find)
            if found:
                return True
        elif item == value_to_find:
            return True
    return False

# Sample hierarchical list
hierarchical_list = [1, [2, [3, 4], 5], [6, 7], 8]

# Value to find
value_to_find = A01B

# Search for the value in the hierarchical list
if search_nested_list(hierarchical_list, value_to_find):
    print(f"The value {value_to_find} is in the hierarchical list.")
else:
    print(f"The value {value_to_find} is not in the hierarchical list.")


In [ ]:
msg="Hello World"

In [ ]:
msg